In [36]:
# =============================================================================
# DATA LOADING AND NORMALIZATION
# =============================================================================
# Run this cell at the beginning to load and normalize all data.
# All downstream code will work without changes.

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored

# -----------------------------------------------------------------------------
# 1. LOAD YALE CELL-TYPE PROPORTION DATA (no normalization needed - already 0-1)
# -----------------------------------------------------------------------------
yale_stroma_df = pd.read_csv("../data/Stroma_Cell_Prop_4.csv", index_col=0)
yale_tumor_df = pd.read_csv("../data/Tumor_Cell_Prop_4.csv", index_col=0)

# -----------------------------------------------------------------------------
# 2. DEFINE CELL TYPE COLUMNS
# -----------------------------------------------------------------------------
cell_type_cols = [
    'Unidentified', 'Stroma', 'Vessels', 'Tumour', 
    'Proliferating_Tumour', 'Granulocytes', 'M1_mac', 'M2_mac',
    'Exhausted CD8', 'Cytotoxic CD8', 'CD8_cells', 'Treg', 
    'CD4_TFH', 'CD4', 'B_Cells', 'DCs'
]

resistance_cells = ['Proliferating_Tumour', 'Granulocytes', 'Vessels']
response_cells = ['M1_mac', 'M2_mac', 'CD4']

# -----------------------------------------------------------------------------
# 3. FILTER YALE CELL-TYPE DATA
# -----------------------------------------------------------------------------
yale_tumor = yale_tumor_df.dropna(subset=resistance_cells).copy()
yale_tumor = yale_tumor.dropna(subset=["PFS_5Years_months", "PFS_5Years_Index"])

yale_stroma = yale_stroma_df.dropna(subset=response_cells).copy()
yale_stroma = yale_stroma.dropna(subset=["PFS_5Years_months", "PFS_5Years_Index"])

# -----------------------------------------------------------------------------
# 4. LOAD UQ DATA
# -----------------------------------------------------------------------------
uq_tumor_df = pd.read_csv("../data/UQ_CK_large_PFS_OS_only_1st_2nd_line.csv", index_col=0)
uq_stroma_df = pd.read_csv("../data/UQ_Stroma_large_PFS_OS_only_1st_2nd_line.csv", index_col=0)

uq_tumor = uq_tumor_df.dropna(subset=["PFS_5Years_months", "PFS_5Years_index"]).copy()
uq_stroma = uq_stroma_df.dropna(subset=["PFS_5Years_months", "PFS_5Years_index"]).copy()

# -----------------------------------------------------------------------------
# 5. LOAD GENE SIGNATURE LISTS
# -----------------------------------------------------------------------------
gran_genes = pd.read_csv("../data/Granulocytes_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()
vessel_genes = pd.read_csv("../data/Endothelial_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()
tumor_genes = pd.read_csv("../data/Malignant_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()
cd4_genes = pd.read_csv("../data/CD4_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()
m1_genes = pd.read_csv("../data/Macrophages.M1_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()
m2_genes = pd.read_csv("../data/Macrophages.M2_cell_genes.csv", index_col=0).iloc[:, 0].dropna().astype(str).tolist()

# Specific gene signatures for RSF
resistance_genes = ["KRT7", "KRT18", "EFNA1", "SERINC2", "FZD6", "CD24", "CCND3", "S100A9"]
response_genes = ["SIGLEC1", "TLR2", "CXCL9", "CD81", "MRC1", "CCL8", "CCL13", "FCGR1A"]

# -----------------------------------------------------------------------------
# 6. LOAD YALE GENE EXPRESSION DATA
# -----------------------------------------------------------------------------
yale_gene_tumor_df = pd.read_csv("../data/CK_av_3B.csv", index_col=0)
yale_gene_stroma_df = pd.read_csv("../data/Stroma_av_3B.csv", index_col=0)

# -----------------------------------------------------------------------------
# 7. NORMALIZE YALE GENE EXPRESSION (z-score within dataset)
# -----------------------------------------------------------------------------
# Get all gene columns (exclude any clinical columns if present)
yale_tumor_gene_cols = [c for c in yale_gene_tumor_df.columns if c not in ['Response', 'PFS_Days', 'PFS_Index', 'PFS_5Years_months', 'PFS_5Years_Index']]
yale_stroma_gene_cols = [c for c in yale_gene_stroma_df.columns if c not in ['Response', 'PFS_Days', 'PFS_Index', 'PFS_5Years_months', 'PFS_5Years_Index']]

# Z-score normalize Yale gene expression
scaler_yale_tumor = StandardScaler()
yale_gene_tumor_df[yale_tumor_gene_cols] = scaler_yale_tumor.fit_transform(yale_gene_tumor_df[yale_tumor_gene_cols])

scaler_yale_stroma = StandardScaler()
yale_gene_stroma_df[yale_stroma_gene_cols] = scaler_yale_stroma.fit_transform(yale_gene_stroma_df[yale_stroma_gene_cols])

# -----------------------------------------------------------------------------
# 8. NORMALIZE UQ GENE EXPRESSION (z-score within dataset)
# -----------------------------------------------------------------------------
# Helper function to identify numeric columns only
def get_numeric_gene_cols(df):
    """Return columns that are numeric and likely gene expression data."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Exclude known clinical/metadata columns
    exclude_patterns = ['PFS', 'OS', 'Response', 'Histology', 'TMA', 'Patient', 
                        'Age', 'Stage', 'Sex', 'Index', 'Days', 'months', 'Years']
    gene_cols = [c for c in numeric_cols 
                 if not any(pat.lower() in c.lower() for pat in exclude_patterns)]
    return gene_cols

uq_tumor_gene_cols = get_numeric_gene_cols(uq_tumor)
uq_stroma_gene_cols = get_numeric_gene_cols(uq_stroma)

# Z-score normalize UQ gene expression
scaler_uq_tumor = StandardScaler()
uq_tumor[uq_tumor_gene_cols] = scaler_uq_tumor.fit_transform(uq_tumor[uq_tumor_gene_cols])

scaler_uq_stroma = StandardScaler()
uq_stroma[uq_stroma_gene_cols] = scaler_uq_stroma.fit_transform(uq_stroma[uq_stroma_gene_cols])

# -----------------------------------------------------------------------------
# 9. MERGE YALE GENE EXPRESSION WITH CLINICAL DATA
# -----------------------------------------------------------------------------
ytma_471 = yale_tumor_df[yale_tumor_df['TMA'] == 'YTMA_471'].copy()
ytma_471['Patient_ID'] = pd.to_numeric(ytma_471['Patient_ID'], errors='coerce')
ytma_471 = ytma_471.dropna(subset=['Patient_ID'])
ytma_471['Patient_ID'] = ytma_471['Patient_ID'].astype(int)

clinical_cols = ['Patient_ID', 'Response', 'PFS_Days', 'PFS_Index', 
                 'PFS_5Years_months', 'PFS_5Years_Index']
clinical = ytma_471[clinical_cols].drop_duplicates(subset='Patient_ID').set_index('Patient_ID')

yale_tumor_merged = yale_gene_tumor_df.join(clinical, how='inner')
yale_stroma_merged = yale_gene_stroma_df.join(clinical, how='inner')

# -----------------------------------------------------------------------------
# 10. FILTER FOR SPECIFIC GENE SIGNATURES
# -----------------------------------------------------------------------------
yale_gene_tumor = yale_tumor_merged.dropna(subset=resistance_genes).copy()
yale_gene_tumor = yale_gene_tumor.dropna(subset=["PFS_5Years_months", "PFS_5Years_Index"])

yale_gene_stroma = yale_stroma_merged.dropna(subset=response_genes).copy()
yale_gene_stroma = yale_gene_stroma.dropna(subset=["PFS_5Years_months", "PFS_5Years_Index"])

# -----------------------------------------------------------------------------
# VERIFICATION
# -----------------------------------------------------------------------------
print("=== Data Loading & Normalization Complete ===\n")
print("YALE Cell-Type Data:")
print(f"  yale_tumor: {yale_tumor.shape[0]} samples")
print(f"  yale_stroma: {yale_stroma.shape[0]} samples")

print("\nYALE Gene Expression (normalized):")
print(f"  yale_gene_tumor: {yale_gene_tumor.shape[0]} samples")
print(f"  yale_gene_stroma: {yale_gene_stroma.shape[0]} samples")
print(f"  Resistance genes mean: {yale_gene_tumor[resistance_genes].mean().mean():.3f} (should be ~0)")
print(f"  Resistance genes std:  {yale_gene_tumor[resistance_genes].std().mean():.3f} (should be ~1)")

print("\nUQ Data (normalized):")
print(f"  uq_tumor: {uq_tumor.shape[0]} samples, {len(uq_tumor_gene_cols)} gene columns")
print(f"  uq_stroma: {uq_stroma.shape[0]} samples, {len(uq_stroma_gene_cols)} gene columns")
if all(g in uq_tumor.columns for g in resistance_genes):
    print(f"  Resistance genes mean: {uq_tumor[resistance_genes].mean().mean():.3f} (should be ~0)")
    print(f"  Resistance genes std:  {uq_tumor[resistance_genes].std().mean():.3f} (should be ~1)")

print("\n✓ All data normalized - RSF models will generalize across datasets")

=== Data Loading & Normalization Complete ===

YALE Cell-Type Data:
  yale_tumor: 67 samples
  yale_stroma: 68 samples

YALE Gene Expression (normalized):
  yale_gene_tumor: 26 samples
  yale_gene_stroma: 26 samples
  Resistance genes mean: 0.054 (should be ~0)
  Resistance genes std:  0.937 (should be ~1)

UQ Data (normalized):
  uq_tumor: 30 samples, 1806 gene columns
  uq_stroma: 27 samples, 1806 gene columns
  Resistance genes mean: 0.000 (should be ~0)
  Resistance genes std:  1.017 (should be ~1)

✓ All data normalized - RSF models will generalize across datasets


In [37]:
def make_survival_y(df, time_col, event_col):
    """
    Convert time + event columns into sksurv-compatible structured array.
    """
    times = df[time_col].to_numpy(dtype=float)
    events = df[event_col].to_numpy(dtype=int)
    y = np.array(
        [(bool(e), t) for e, t in zip(events, times)],
        dtype=[("event", bool), ("time", float)],
    )
    return y

In [38]:
from sklearn.inspection import permutation_importance
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored


def cv_rsf_cindex(X, y, feature_names=None, n_splits=5, rsf_params=None, random_state=0):
    """
    Run K-fold CV for RSF, compute mean/std C-index, and fit RSF on full dataset.
    
    Parameters
    ----------
    X : np.ndarray
        Feature matrix (n_samples, n_features).
    y : structured array
        Survival targets with fields ("event", "time").
    feature_names : list or None
        Names of features. If None, uses indices.
    n_splits : int
        Number of CV folds.
    rsf_params : dict or None
        Parameters for RandomSurvivalForest.
    random_state : int
        RNG seed for reproducibility.
    
    Returns
    -------
    mean_cindex : float
    std_cindex : float
    rsf_full : RandomSurvivalForest fitted on all data.
    feature_importances : pd.DataFrame
        Features sorted by importance (descending).
    """
    if rsf_params is None:
        rsf_params = {
            "n_estimators": 1000,
            "min_samples_split": 10,
            "min_samples_leaf": 5,
            "max_features": "sqrt",
            "n_jobs": -1,
            "random_state": random_state,
        }
    
    if feature_names is None:
        feature_names = [f"feature_{i}" for i in range(X.shape[1])]
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    c_indices = []
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        rsf = RandomSurvivalForest(**rsf_params)
        rsf.fit(X[train_idx], y[train_idx])
        
        ci = concordance_index_censored(
            y[test_idx]["event"], y[test_idx]["time"], rsf.predict(X[test_idx])
        )[0]
        print(f"Fold {fold}: C-index = {ci:.3f}")
        c_indices.append(ci)
    
    c_indices = np.array(c_indices)
    
    # Fit on full dataset
    rsf_full = RandomSurvivalForest(**rsf_params)
    rsf_full.fit(X, y)
    print("\nFitted final RSF model on all samples.")
    
    # Calculate permutation importance
    perm_imp = permutation_importance(
        rsf_full, X, y, 
        n_repeats=10, 
        random_state=random_state, 
        n_jobs=-1
    )
    
    feature_importances = pd.DataFrame({
        "feature": feature_names,
        "importance": perm_imp.importances_mean,
        "std": perm_imp.importances_std
    }).sort_values("importance", ascending=False).reset_index(drop=True)
    
    return c_indices.mean(), c_indices.std(), rsf_full, feature_importances

In [39]:
# Training RSF for cell-signature resistance
X_res = yale_tumor[resistance_cells].astype(float).to_numpy()
y_res = make_survival_y(yale_tumor, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance signature (3 features)...")
mean_ci_res, std_ci_res, rsf_cell_resistance, _ = cv_rsf_cindex(
    X_res, y_res, feature_names=resistance_cells, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

Training RSF on resistance signature (3 features)...
Fold 1: C-index = 0.538
Fold 2: C-index = 0.370
Fold 3: C-index = 0.640
Fold 4: C-index = 0.441
Fold 5: C-index = 0.676

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.533 ± 0.116


In [40]:
from lifelines import CoxPHFitter

def evaluate_rsf(rsf, X, y, cut="median", invert_risk=False):
    """
    Use trained RSF to compute risk scores and fit Cox model for HR, 95% CI, and p-value.
    
    Parameters
    ----------
    rsf : fitted RandomSurvivalForest
    X : np.ndarray
        Feature matrix for evaluation cohort.
    y : structured array
        Survival targets with fields ("event", "time").
    cut : {"median", "tertile"} or float
        Threshold for binary risk grouping.
    protective : bool
        If True, compare high vs low feature expression (for protective signatures).
        If False, compare high vs low RSF risk (for risk signatures).
    
    Returns
    -------
    hr : float
    ci : tuple (lower, upper)
    p_value : float
    cph : CoxPHFitter
    risk_scores : np.ndarray
    """
    risk_scores = rsf.predict(X)
    
    # Determine threshold
    if cut == "median":
        threshold = np.median(risk_scores)
    elif cut == "tertile":
        threshold = np.percentile(risk_scores, 67)
    else:
        threshold = float(cut)
    
    # For protective signatures, flip the grouping
    # high RSF risk = low feature expression, so we invert to get "high expression" group
    if invert_risk:
        high_group = (risk_scores < threshold).astype(int)  # high expression = low RSF risk
    else:
        high_group = (risk_scores >= threshold).astype(int)  # high risk
    
    df = pd.DataFrame({
        "time": y["time"],
        "event": y["event"].astype(int),
        "high_group": high_group,
    })
    
    cph = CoxPHFitter()
    cph.fit(df, duration_col="time", event_col="event")
    
    summary = cph.summary
    hr = summary.loc["high_group", "exp(coef)"]

    ci = (summary.loc["high_group", "exp(coef) lower 95%"],
        summary.loc["high_group", "exp(coef) upper 95%"])
    
    p_value = summary.loc["high_group", "p"]
    
    return hr, ci, p_value, cph, risk_scores

In [41]:
# evaluating cell signature resistance rsf on yale cohort
yale_cell_res_hr, yale_cell_res_ci, yale_cell_res_p, cph, scores = evaluate_rsf(
    rsf_cell_resistance, X_res, y_res, 
)

print(f"HR (high vs low RSF risk, median split): {yale_cell_res_hr:.2f}")
print(f"95% CI: [{yale_cell_res_ci[0]:.2f}, {yale_cell_res_ci[1]:.2f}]")

HR (high vs low RSF risk, median split): 5.24
95% CI: [2.73, 10.06]


In [42]:
# yale tumor rsf for all features
X_res = yale_tumor[cell_type_cols].astype(float).to_numpy()
y_res = make_survival_y(yale_tumor, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance signature (all features)...")
mean_ci_res, std_ci_res, rsf_cell_resistance_all, yale_c_t_fi = cv_rsf_cindex(
    X_res, y_res, feature_names=cell_type_cols, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

Training RSF on resistance signature (all features)...
Fold 1: C-index = 0.462
Fold 2: C-index = 0.370
Fold 3: C-index = 0.691
Fold 4: C-index = 0.544
Fold 5: C-index = 0.691

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.552 ± 0.127


In [43]:
# evaluating cell signature rsf on yale cohort all cell types
yale_cell_res_hr_all, yale_cell_res_ci_all, yale_cell_res_p_all, cph, scores = evaluate_rsf(
    rsf_cell_resistance_all, X_res, y_res, 
)

print(f"HR (high vs low RSF risk, median split): {yale_cell_res_hr_all:.2f}")
print(f"95% CI: [{yale_cell_res_ci_all[0]:.2f}, {yale_cell_res_ci_all[1]:.2f}]")

HR (high vs low RSF risk, median split): 8.25
95% CI: [4.16, 16.37]


In [44]:
# training RSF on response signature
X_resp = yale_stroma[response_cells].astype(float).to_numpy()
y_resp = make_survival_y(yale_stroma, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance signature (3 features)...")
mean_ci_res, std_ci_res, rsf_cell_response, _ = cv_rsf_cindex(
    X_resp, y_resp, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 5-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

# evaluating cell signature response rsf on yale cohort
yale_cell_resp_hr, yale_cell_resp_ci, yale_cell_resp_p, cph, scores = evaluate_rsf(
    rsf_cell_response, X_resp, y_resp, invert_risk=True
)

print(f"HR (high vs low RSF risk, median split): {yale_cell_resp_hr:.2f}")
print(f"95% CI: [{yale_cell_resp_ci[0]:.2f}, {yale_cell_resp_ci[1]:.2f}]")

Training RSF on resistance signature (3 features)...
Fold 1: C-index = 0.625
Fold 2: C-index = 0.448
Fold 3: C-index = 0.457
Fold 4: C-index = 0.525
Fold 5: C-index = 0.483

Fitted final RSF model on all samples.

RSF (resistance signature) 5-fold C-index: 0.508 ± 0.064
HR (high vs low RSF risk, median split): 0.18
95% CI: [0.09, 0.36]


In [45]:
# training cell signature response rsf (stroma compartment) all cell types
X_resp = yale_stroma[cell_type_cols].astype(float).to_numpy()
print("Training RSF on resistance signature (all features)...")
mean_ci_res, std_ci_res, rsf_cell_response_all, yale_c_s_fi = cv_rsf_cindex(
    X_resp, y_resp, feature_names=cell_type_cols, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

# evaluating cell signature response rsf on yale cohort
yale_cell_resp_hr_all, yale_cell_resp_ci_all, yale_cell_resp_p_all, cph, scores = evaluate_rsf(
    rsf_cell_response_all, X_resp, y_resp, invert_risk=True
)

print(f"HR (high vs low RSF risk, median split): {yale_cell_resp_hr_all:.2f}")
print(f"95% CI: [{yale_cell_resp_ci_all[0]:.2f}, {yale_cell_resp_ci_all[1]:.2f}]")

Training RSF on resistance signature (all features)...
Fold 1: C-index = 0.761
Fold 2: C-index = 0.575
Fold 3: C-index = 0.629
Fold 4: C-index = 0.593
Fold 5: C-index = 0.810

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.674 ± 0.095
HR (high vs low RSF risk, median split): 0.04
95% CI: [0.02, 0.12]


In [46]:
def compute_composite_score(
    df,
    feature_list,
    min_features=2,
    zscore=True,
    prefix=None,
    verbose=True,
):
    """
    Compute a composite signature score from a list of gene/protein features.

    Parameters
    ----------
    df : pd.DataFrame
        Input data with samples as rows and genes/proteins as columns.
    feature_list : list of str
        List of gene or protein feature names defining the signature.
    min_features : int, default=2
        Minimum number of features required to compute the score.
    zscore : bool, default=True
        Whether to z-score each feature across samples before aggregation.
    prefix : str or None
        Optional name prefix for the returned Series.
    verbose : bool, default=True
        Print warnings about missing features.

    Returns
    -------
    score : pd.Series
        Composite score indexed like df.index.

    Raises
    ------
    ValueError
        If fewer than min_features are found in df.
    """
    # 1. Keep only features present in the dataset
    present = [f for f in feature_list if f in df.columns]

    if verbose:
        print(f"Found {len(present)} / {len(feature_list)} features.")

    if len(present) < min_features:
        raise ValueError(
            f"Only {len(present)} features found, "
            f"but min_features={min_features} required."
        )

    X = df[present].astype(float)

    # 2. Z-score each feature across samples
    if zscore:
        X = (X - X.mean(axis=0)) / (X.std(axis=0, ddof=0) + 1e-8)

    # 3. Composite score = mean across features
    score = X.mean(axis=1)

    if prefix is not None:
        score.name = prefix
    else:
        score.name = "composite_score"

    return score


In [47]:
# aggregating relevant genes into a z-score for their corresponding cell-type category
uq_tumor["Proliferating_Tumour"] = compute_composite_score(uq_tumor, tumor_genes)
uq_tumor["Vessels"] = compute_composite_score(uq_tumor, vessel_genes)
uq_tumor["Granulocytes"] = compute_composite_score(uq_tumor, gran_genes)

uq_stroma["M1_mac"] = compute_composite_score(uq_stroma, m1_genes)
uq_stroma["M2_mac"] = compute_composite_score(uq_stroma, m2_genes)
uq_stroma["CD4"] = compute_composite_score(uq_stroma, cd4_genes)


Found 315 / 323 features.
Found 297 / 309 features.
Found 187 / 187 features.
Found 143 / 143 features.
Found 186 / 186 features.
Found 199 / 199 features.


C:\Users\blued\AppData\Local\Temp\ipykernel_31856\3135294284.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  uq_tumor["Proliferating_Tumour"] = compute_composite_score(uq_tumor, tumor_genes)
C:\Users\blued\AppData\Local\Temp\ipykernel_31856\3135294284.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  uq_tumor["Vessels"] = compute_composite_score(uq_tumor, vessel_genes)
C:\Users\blued\AppData\Local\Temp\ipykernel_31856\3135294284.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin

In [48]:
X_res = uq_tumor[resistance_cells].astype(float).to_numpy()
y_res = make_survival_y(uq_tumor, "PFS_5Years_months", "PFS_5Years_index")

# UQ resistance validation
uq_cell_res_hr, uq_cell_res_ci, uq_cell_res_p, cph, scores = evaluate_rsf(
    rsf_cell_resistance, X_res, y_res
)

print(f"HR (high vs low RSF risk, median split): {uq_cell_res_hr:.2f}")
print(f"95% CI: [{uq_cell_res_ci[0]:.2f}, {uq_cell_res_ci[1]:.2f}]")

HR (high vs low RSF risk, median split): 0.95
95% CI: [0.44, 2.07]


In [49]:
X_resp = uq_stroma[response_cells].astype(float).to_numpy()
y_resp = make_survival_y(uq_stroma, "PFS_5Years_months", "PFS_5Years_index")

# UQ resistance validation all features
uq_cell_resp_hr, uq_cell_resp_ci, uq_cell_resp_p, cph, scores = evaluate_rsf(
    rsf_cell_response, X_resp, y_resp, invert_risk=True
)

print(f"HR (high vs low RSF risk, median split): {uq_cell_resp_hr:.2f}")
print(f"95% CI: [{uq_cell_resp_ci[0]:.2f}, {uq_cell_resp_ci[1]:.2f}]")

HR (high vs low RSF risk, median split): 0.99
95% CI: [0.43, 2.28]


In [50]:
# training RSF on gene resistance
X_res = yale_gene_tumor[resistance_genes].astype(float).to_numpy()
y_res = make_survival_y(yale_gene_tumor, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance gene signature (8 features)...")
mean_ci_res, std_ci_res, rsf_gene_resistance, _ = cv_rsf_cindex(
    X_res, y_res, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

# X_res and y are from Yale resistance cohort
yale_gene_res_hr, yale_gene_res_ci, yale_gene_res_p, cph, scores = evaluate_rsf(
    rsf_gene_resistance, X_res, y_res
)

print(f"HR (high vs low RSF risk, median split): {yale_gene_res_hr:.2f}")
print(f"95% CI: [{yale_gene_res_ci[0]:.2f}, {yale_gene_res_ci[1]:.2f}]")

Training RSF on resistance gene signature (8 features)...
Fold 1: C-index = 0.500
Fold 2: C-index = 0.600
Fold 3: C-index = 0.900
Fold 4: C-index = 0.571
Fold 5: C-index = 0.400

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.594 ± 0.168
HR (high vs low RSF risk, median split): 5.19
95% CI: [1.71, 15.81]


In [51]:
# UQ validation on resistance genes rsf
X_res = uq_tumor[resistance_genes].astype(float).to_numpy()
y_res = make_survival_y(uq_tumor, "PFS_5Years_months", "PFS_5Years_index")

uq_gene_res_hr, uq_gene_res_ci, uq_gene_res_p, cph, scores = evaluate_rsf(
    rsf_gene_resistance, X_res, y_res
)

print(f"HR (high vs low RSF risk, median split): {uq_gene_res_hr:.2f}")
print(f"95% CI: [{uq_gene_res_ci[0]:.2f}, {uq_gene_res_ci[1]:.2f}]")

HR (high vs low RSF risk, median split): 1.67
95% CI: [0.74, 3.75]


In [52]:
# gene list present in both yale and uq datasets
yale_genes = yale_gene_tumor.iloc[:, :-5].columns.tolist()
gene_list = [c for c in uq_tumor.columns if c in yale_genes]

In [53]:
# training RSF on gene resistance with all features
X_res = yale_gene_tumor[gene_list].astype(float).to_numpy()
y_res = make_survival_y(yale_gene_tumor, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance gene signature (all features)...")
mean_ci_res, std_ci_res, rsf_gene_resistance_all, yale_g_t_fi = cv_rsf_cindex(
    X_res, y_res, feature_names=gene_list, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_res:.3f} ± {std_ci_res:.3f}")

# X_res and y are from Yale resistance cohort
yale_gene_res_hr_all, yale_gene_res_ci_all, yale_gene_res_p_all, cph, scores = evaluate_rsf(
    rsf_gene_resistance_all, X_res, y_res
)

print(f"HR (high vs low RSF risk, median split): {yale_gene_res_hr_all:.2f}")
print(f"95% CI: [{yale_gene_res_ci_all[0]:.2f}, {yale_gene_res_ci_all[1]:.2f}]")

Training RSF on resistance gene signature (all features)...
Fold 1: C-index = 0.429
Fold 2: C-index = 0.400
Fold 3: C-index = 0.800
Fold 4: C-index = 0.429
Fold 5: C-index = 0.400

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.491 ± 0.155
HR (high vs low RSF risk, median split): 262300561.02
95% CI: [0.00, inf]


c:\Users\blued\miniconda3\envs\genomic_info\lib\site-packages\lifelines\utils\__init__.py:1120: ConvergenceWarning: Column high_group have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'high_group'].var())
>>> print(df.loc[~events, 'high_group'].var())

A very low variance means that the column high_group completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
c:\Users\blued\miniconda3\envs\genomic_info\lib\site-packages\lifelines\fitters\coxph_fitter.py:1607: ConvergenceWarning: Newton-Raphson convergence completed successfully but norm(delta) is still high, 0.510. This may imply non-unique solutions to the maximum likelihood. P

In [54]:
# UQ validation on resistance genes with all genes
X_res = uq_tumor[gene_list].astype(float).to_numpy()
y_res = make_survival_y(uq_tumor, "PFS_5Years_months", "PFS_5Years_index")

uq_gene_res_hr_all, uq_gene_res_ci_all, uq_gene_res_p_all, cph, scores = evaluate_rsf(
    rsf_gene_resistance_all, X_res, y_res
)

print(f"HR (high vs low RSF risk, median split): {uq_gene_res_hr_all:.2f}")
print(f"95% CI: [{uq_gene_res_ci_all[0]:.2f}, {uq_gene_res_ci_all[1]:.2f}]")

HR (high vs low RSF risk, median split): 1.76
95% CI: [0.81, 3.85]


In [55]:
# training RSF on gene response
X_resp = yale_gene_stroma[response_genes].astype(float).to_numpy()
y_resp = make_survival_y(yale_gene_stroma, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance gene signature (8 features)...")
mean_ci_resp, std_ci_resp, rsf_gene_response, _ = cv_rsf_cindex(
    X_resp, y_resp, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_resp:.3f} ± {std_ci_resp:.3f}")

# X_resp and y are from Yale response cohort
yale_gene_resp_hr, yale_gene_resp_ci, yale_gene_resp_p, cph, scores = evaluate_rsf(
    rsf_gene_response, X_resp, y_resp, invert_risk=True
)

print(f"HR (high vs low RSF risk, median split): {yale_gene_resp_hr:.2f}")
print(f"95% CI: [{yale_gene_resp_ci[0]:.2f}, {yale_gene_resp_ci[1]:.2f}]")

Training RSF on resistance gene signature (8 features)...
Fold 1: C-index = 0.750
Fold 2: C-index = 1.000
Fold 3: C-index = 0.375
Fold 4: C-index = 0.857
Fold 5: C-index = 0.100

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.616 ± 0.331
HR (high vs low RSF risk, median split): 0.03
95% CI: [0.00, 0.23]


In [56]:
# UQ validation on gene response
X_resp = uq_stroma[response_genes].astype(float).to_numpy()
y_resp = make_survival_y(uq_stroma, "PFS_5Years_months", "PFS_5Years_index")

uq_gene_resp_hr, uq_gene_resp_ci, uq_gene_resp_p, cph, scores = evaluate_rsf(
    rsf_gene_response, X_resp, y_resp, 
)

print(f"HR (high vs low RSF risk, median split): {uq_gene_resp_hr:.2f}")
print(f"95% CI: [{uq_gene_resp_ci[0]:.2f}, {uq_gene_resp_ci[1]:.2f}]")

HR (high vs low RSF risk, median split): 0.90
95% CI: [0.39, 2.04]


In [57]:
# gene list present in both yale and uq datasets
yale_stroma_genes = yale_gene_stroma.iloc[:, :-5].columns.tolist()
gene_list_stroma = [c for c in uq_stroma.columns if c in yale_stroma_genes]

In [58]:
# training RSF on gene response all features
X_resp = yale_gene_stroma[gene_list_stroma].astype(float).to_numpy()
y_resp = make_survival_y(yale_gene_stroma, "PFS_5Years_months", "PFS_5Years_Index")
print("Training RSF on resistance gene signature (all features)...")
mean_ci_resp_all, std_ci_resp_all, rsf_gene_response_all, yale_g_s_fi = cv_rsf_cindex(
    X_resp, y_resp, feature_names=gene_list_stroma, n_splits=5, random_state=0
)

print(f"\nRSF (resistance signature) 10-fold C-index: {mean_ci_resp_all:.3f} ± {std_ci_resp_all:.3f}")

# X_resp and y are from Yale response cohort
yale_gene_resp_hr_all, yale_gene_resp_ci_all, yale_gene_resp_p_all, cph, scores = evaluate_rsf(
    rsf_gene_response_all, X_resp, y_resp, invert_risk=True
)

print(f"HR (high vs low RSF risk, median split): {yale_gene_resp_hr_all:.2f}")
print(f"95% CI: [{yale_gene_resp_ci_all[0]:.2f}, {yale_gene_resp_ci_all[1]:.2f}]")

Training RSF on resistance gene signature (all features)...
Fold 1: C-index = 0.333
Fold 2: C-index = 0.333
Fold 3: C-index = 0.625
Fold 4: C-index = 0.714
Fold 5: C-index = 0.600

Fitted final RSF model on all samples.

RSF (resistance signature) 10-fold C-index: 0.521 ± 0.158
HR (high vs low RSF risk, median split): 0.02
95% CI: [0.00, 0.19]


In [59]:
print(len(gene_list), len(gene_list_stroma))

1092 844


In [60]:
# UQ validation on gene response for all features
X_resp = uq_stroma[gene_list_stroma].astype(float).to_numpy()
y_resp = make_survival_y(uq_stroma, "PFS_5Years_months", "PFS_5Years_index")

uq_gene_resp_hr_all, uq_gene_resp_ci_all, uq_gene_resp_p_all, cph, scores = evaluate_rsf(
    rsf_gene_response_all, X_resp, y_resp, 
)

print(f"HR (high vs low RSF risk, median split): {uq_gene_resp_hr_all:.2f}")
print(f"95% CI: [{uq_gene_resp_ci_all[0]:.2f}, {uq_gene_resp_ci_all[1]:.2f}]")

HR (high vs low RSF risk, median split): 0.54
95% CI: [0.23, 1.24]


In [61]:
# =============================================================================
# RESULTS SUMMARY TABLE
# =============================================================================

# Create results dictionary
results = []

# -----------------------------------------------------------------------------
# CELL-TYPE SIGNATURES
# -----------------------------------------------------------------------------
# Yale Resistance (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Resistance',
    'Cohort': 'Yale (Training)',
    'HR': yale_cell_res_hr,
    'CI_lower': yale_cell_res_ci[0],
    'CI_upper': yale_cell_res_ci[1],
    'p_value': yale_cell_res_p
})

# Yale Response (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Response',
    'Cohort': 'Yale (Training)',
    'HR': yale_cell_resp_hr,
    'CI_lower': yale_cell_resp_ci[0],
    'CI_upper': yale_cell_resp_ci[1],
    'p_value': yale_cell_resp_p
})

# Yale Resistance All Features (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Resistance All Features',
    'Cohort': 'Yale (Training)',
    'HR': yale_cell_res_hr_all,
    'CI_lower': yale_cell_res_ci_all[0],
    'CI_upper': yale_cell_res_ci_all[1],
    'p_value': yale_cell_res_p_all
})

# Yale Response All Features (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Response All Features',
    'Cohort': 'Yale (Training)',
    'HR': yale_cell_resp_hr_all,
    'CI_lower': yale_cell_resp_ci_all[0],
    'CI_upper': yale_cell_resp_ci_all[1],
    'p_value': yale_cell_resp_p_all
})

# UQ Resistance (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Resistance',
    'Cohort': 'UQ (Validation)',
    'HR': uq_cell_res_hr,
    'CI_lower': uq_cell_res_ci[0],
    'CI_upper': uq_cell_res_ci[1],
    'p_value': uq_cell_res_p
})

# UQ Response (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Response',
    'Cohort': 'UQ (Validation)',
    'HR': uq_cell_resp_hr,
    'CI_lower': uq_cell_resp_ci[0],
    'CI_upper': uq_cell_resp_ci[1],
    'p_value': uq_cell_resp_p
})

# UQ Resistance All Features (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Resistance All Features',
    'Cohort': 'UQ (Validation)',
    'HR': uq_cell_res_hr_all,
    'CI_lower': uq_cell_res_ci_all[0],
    'CI_upper': uq_cell_res_ci_all[1],
    'p_value': uq_cell_res_p_all
})

# UQ Response All Features (Cell)
results.append({
    'Signature Type': 'Cell-Type',
    'Signature': 'Response All Features',
    'Cohort': 'UQ (Validation)',
    'HR': uq_cell_resp_hr_all,
    'CI_lower': uq_cell_resp_ci_all[0],
    'CI_upper': uq_cell_resp_ci_all[1],
    'p_value': uq_cell_resp_p_all
})

# -----------------------------------------------------------------------------
# GENE SIGNATURES
# -----------------------------------------------------------------------------
# Yale Resistance (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Resistance',
    'Cohort': 'Yale (Training)',
    'HR': yale_gene_res_hr,
    'CI_lower': yale_gene_res_ci[0],
    'CI_upper': yale_gene_res_ci[1],
    'p_value': yale_gene_res_p
})

# Yale Response (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Response',
    'Cohort': 'Yale (Training)',
    'HR': yale_gene_resp_hr,
    'CI_lower': yale_gene_resp_ci[0],
    'CI_upper': yale_gene_resp_ci[1],
    'p_value': yale_gene_resp_p
})

# Yale Resistance All Features (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Resistance All Features',
    'Cohort': 'Yale (Training)',
    'HR': yale_gene_res_hr_all,
    'CI_lower': yale_gene_res_ci_all[0],
    'CI_upper': yale_gene_res_ci_all[1],
    'p_value': yale_gene_res_p_all
})

# Yale Response All Features (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Response All Features',
    'Cohort': 'Yale (Training)',
    'HR': yale_gene_resp_hr_all,
    'CI_lower': yale_gene_resp_ci_all[0],
    'CI_upper': yale_gene_resp_ci_all[1],
    'p_value': yale_gene_resp_p_all
})

# UQ Resistance (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Resistance',
    'Cohort': 'UQ (Validation)',
    'HR': uq_gene_res_hr,
    'CI_lower': uq_gene_res_ci[0],
    'CI_upper': uq_gene_res_ci[1],
    'p_value': uq_gene_res_p
})

# UQ Response (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Response',
    'Cohort': 'UQ (Validation)',
    'HR': uq_gene_resp_hr,
    'CI_lower': uq_gene_resp_ci[0],
    'CI_upper': uq_gene_resp_ci[1],
    'p_value': uq_gene_resp_p
})

# UQ Resistance All Features (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Resistance All Features',
    'Cohort': 'UQ (Validation)',
    'HR': uq_gene_res_hr_all,
    'CI_lower': uq_gene_res_ci_all[0],
    'CI_upper': uq_gene_res_ci_all[1],
    'p_value': uq_gene_res_p_all
})

# UQ Response All Features (Gene)
results.append({
    'Signature Type': 'Gene',
    'Signature': 'Response All Features',
    'Cohort': 'UQ (Validation)',
    'HR': uq_gene_resp_hr_all,
    'CI_lower': uq_gene_resp_ci_all[0],
    'CI_upper': uq_gene_resp_ci_all[1],
    'p_value': uq_gene_resp_p_all
})

# -----------------------------------------------------------------------------
# CREATE FORMATTED TABLE
# -----------------------------------------------------------------------------
results_df = pd.DataFrame(results)

# Format columns for display
results_df['HR (95% CI)'] = results_df.apply(
    lambda x: f"{x['HR']:.2f} ({x['CI_lower']:.2f}-{x['CI_upper']:.2f})", axis=1
)
results_df['p-value'] = results_df['p_value'].apply(
    lambda x: f"{x:.3f}" if x >= 0.001 else "<0.001"
)

# Create clean display table
display_df = results_df.pivot_table(
    index=['Signature Type', 'Signature'],
    columns='Cohort',
    values=['HR (95% CI)', 'p-value'],
    aggfunc='first'
)

# Reorder columns
display_df = display_df.reindex(columns=[
    ('HR (95% CI)', 'Yale (Training)'),
    ('p-value', 'Yale (Training)'),
    ('HR (95% CI)', 'UQ (Validation)'),
    ('p-value', 'UQ (Validation)')
])

# Reorder rows to have consistent order
row_order = [
    ('Cell-Type', 'Resistance'),
    ('Cell-Type', 'Response'),
    ('Cell-Type', 'Resistance All Features'),
    ('Cell-Type', 'Response All Features'),
    ('Gene', 'Resistance'),
    ('Gene', 'Response'),
    ('Gene', 'Resistance All Features'),
    ('Gene', 'Response All Features'),
]
display_df = display_df.reindex(row_order)

print("=" * 100)
print("RSF SURVIVAL ANALYSIS RESULTS: High vs Low Risk (Median Split)")
print("=" * 100)
print(display_df.to_string())

# Also create a simpler flat table for export
export_df = results_df[['Signature Type', 'Signature', 'Cohort', 'HR (95% CI)', 'p-value']].copy()
export_df = export_df.sort_values(['Signature Type', 'Signature', 'Cohort'])

print("\n" + "=" * 100)
print("FLAT FORMAT:")
print("=" * 100)
print(export_df.to_string(index=False))

# Save to CSV
results_df.to_csv("rsf_results_summary.csv", index=False)
print("\n✓ Results saved to rsf_results_summary.csv")


RSF SURVIVAL ANALYSIS RESULTS: High vs Low Risk (Median Split)
                                                    HR (95% CI)         p-value       HR (95% CI)         p-value
Cohort                                          Yale (Training) Yale (Training)   UQ (Validation) UQ (Validation)
Signature Type Signature                                                                                         
Cell-Type      Resistance                     5.24 (2.73-10.06)          <0.001  0.95 (0.44-2.07)           0.902
               Response                        0.18 (0.09-0.36)          <0.001  0.99 (0.43-2.28)           0.976
               Resistance All Features        8.25 (4.16-16.37)          <0.001  0.95 (0.44-2.07)           0.902
               Response All Features           0.04 (0.02-0.12)          <0.001  0.99 (0.43-2.28)           0.976
Gene           Resistance                     5.19 (1.71-15.81)           0.004  1.67 (0.74-3.75)           0.217
               Response  

In [66]:
# =============================================================================
# TOP FEATURES FROM RSF MODELS (ALL FEATURES)
# =============================================================================

print("=" * 70)
print("CELL-TYPE SIGNATURES - TOP FEATURES")
print("=" * 70)

# Assuming feature importances were saved from cv_rsf_cindex
# Adjust variable names as needed based on your notebook

print("\n--- Cell-Type Resistance (Tumor) ---")
print(yale_c_t_fi.head(10).to_string(index=False))

print("\n--- Cell-Type Response (Stroma) ---")
print(yale_c_s_fi.head(10).to_string(index=False))






CELL-TYPE SIGNATURES - TOP FEATURES

--- Cell-Type Resistance (Tumor) ---
             feature  importance      std
             Vessels    0.033862 0.013678
                 DCs    0.022302 0.008838
                 CD4    0.016667 0.003895
Proliferating_Tumour    0.015556 0.003107
              Tumour    0.013704 0.004371
        Granulocytes    0.011243 0.005862
              Stroma    0.008466 0.009492
                Treg    0.007011 0.007342
              M1_mac    0.006614 0.002504
             CD4_TFH    0.006243 0.006008

--- Cell-Type Response (Stroma) ---
             feature  importance      std
        Unidentified    0.106438 0.018524
                 DCs    0.084839 0.020116
        Granulocytes    0.020509 0.004669
             B_Cells    0.015472 0.009232
              Stroma    0.014901 0.004313
              M1_mac    0.012928 0.003769
                Treg    0.009553 0.004651
              M2_mac    0.006023 0.003817
Proliferating_Tumour    0.005659 0.004131
       

In [67]:
print("\n" + "=" * 70)
print("GENE SIGNATURES - TOP FEATURES")
print("=" * 70)

# Gene Resistance
print("\n--- Gene-Type Resistance (Tumor) ---")
print(yale_g_t_fi.head(10).to_string(index=False))

# Gene Response
print("\n--- Gene-Type Response (Stroma) ---")
print(yale_g_s_fi.head(10).to_string(index=False))


GENE SIGNATURES - TOP FEATURES

--- Gene-Type Resistance (Tumor) ---
feature  importance      std
   ABL1    0.008303 0.005118
 IL1RAP    0.006498 0.002702
    BID    0.005776 0.004332
  ITGB2    0.004332 0.004210
   NSD2    0.003610 0.000000
  CCND3    0.003610 0.002283
   MRM2    0.003249 0.001083
    MX1    0.003249 0.001083
  SMAD2    0.003249 0.001083
  IFNA8    0.003249 0.001083

--- Gene-Type Response (Stroma) ---
feature  importance      std
  SIRPA    0.004000 0.001091
COLEC12    0.004000 0.001958
  NTRK2    0.003636 0.000000
  RPL7A    0.003636 0.000000
  GAGE1    0.003636 0.000000
  C3AR1    0.003636 0.000000
 PSMB10    0.003636 0.000000
   JAK3    0.003636 0.000000
  EFNA1    0.003636 0.000000
  RPLP0    0.003636 0.000000
